In [74]:
# Dynamic Programming: O(n*m), psuedo-polynominal
class KnapsackProblem:

    def __init__(self, n, M, w, v):
        self.n = n
        self.M = M
        self.w = w
        self.v = v
        self.S = [[0 for _ in range(M+1)] for _ in range(n+1)]

    def solve(self):
        # construct the S dynamic programming table
        # O(n*M)
        for i in range(self.n+1):
            for w in range(self.M+1):
                not_taking_item = self.S[i - 1][w]
                taking_item = 0

                if self.w[i] <= w:
                    taking_item = self.v[i] + self.S[i - 1][w - self.w[i]]

                # memoization - we store the sub-results to avoid recalculating the same values
                self.S[i][w] = max(not_taking_item, taking_item)

    def show_result(self):

        print("Total benefit: %d" % self.S[self.n][self.M])

        w = self.M
        for n in range(self.n, 0, -1):
            if self.S[n][w] != 0 and self.S[n][w] != self.S[n - 1][w]:
                print("We take item #%d" % n)
                w = w - self.w[n]
    
    def show_dp_table(self):
        for i in range(self.n+1):
            row = ""
            for w in range(self.M+1):
                row += str(self.S[i][w]) + ' '
            print(row)


In [75]:
num_of_items = 3
knapsack_capacity = 5
weights = [0, 4, 2, 3]
profits = [0, 10, 4, 7]
knapsack = KnapsackProblem(num_of_items, knapsack_capacity, weights, profits)
knapsack.solve()
knapsack.show_result()
knapsack.show_dp_table()

Total benefit: 11
We take item #3
We take item #2
0 0 0 0 0 0 
0 0 0 0 10 10 
0 0 4 4 10 10 
0 0 4 7 10 11 


In [76]:
num_of_items = 4
knapsack_capacity = 7
weights = [0, 1, 3, 4, 5]
profits = [0, 1, 4, 5, 7]
knapsack = KnapsackProblem(num_of_items, knapsack_capacity, weights, profits)
knapsack.solve()
knapsack.show_result()
knapsack.show_dp_table()

Total benefit: 9
We take item #3
We take item #2
0 0 0 0 0 0 0 0 
0 1 1 1 1 1 1 1 
0 1 1 4 5 5 5 5 
0 1 1 4 5 6 6 9 
0 1 1 4 5 7 8 9 


In [80]:
num_of_items = 2
knapsack_capacity = 7
weights = [0, 1, 2]
profits = [0, 1, 2]
knapsack = KnapsackProblem(num_of_items, knapsack_capacity, weights, profits)
knapsack.solve()
knapsack.show_result()
knapsack.show_dp_table()

Total benefit: 3
We take item #2
We take item #1
0 0 0 0 0 0 0 0 
0 1 1 1 1 1 1 1 
0 1 2 3 3 3 3 3 


In [210]:
import typing

class feat_for_reuse():
    def __init__(self, name, size, accesses):
        self.name = name
        self.weight = size  # Weight in the knapsack algorithm
        self.accesses = accesses  # 1st access is the WRITE, the rest of accesses are READ
        self.start = self.accesses[0]
        self.end = self.accesses[-1]
        self.profit = []

        # profit = (Size * # READs) / Total SRAM allocated time
        #self.profit = (size * (len(self.accesses)-1)) / (self.end - self.start + 1)

        if self.start > self.end:
            raise ValueError('start time is larger than end time')        
    
    def is_candidate(self, exe_time):
        if exe_time >= self.start and exe_time <= self.end:
            return True
        return False
    
    def remove_access(self, exe_time):
        if not self.is_candidate(exe_time):
            raise ValueError("exetime is not in the candidate.")
        
        split_accesses = []
        # cut the accesses
        for idx in range(self.accesses, -1, -1):
            if self.access[idx] < exe_time and idx > 1:  # idx == 1 then there is nothing to cut
                split_accesses = self.accesses[0:idx+1]        
        return split_accesses


    def update_accesses(self, split_accesses):
        self.accesses = split_accesses
        self.end = self.accesses[-1]


    def cal_profit(self):
        self.profit = []
        profit = 0
        next_time = self.end + 1
        not_accessed = 0
        last_real_profit = 0

        for exe_time in range(self.end, self.start-1, -1):
            if exe_time in self.accesses[1:]:  # consecutive access
                if not_accessed > 0:  # if prev_access is not real access, reset the profit value
                    not_accessed = 0
                    profit = 0
                profit += self.weight        
                last_real_profit = profit
                self.profit.append(profit)
            else:
                not_accessed += 1                
                self.profit.append(last_real_profit/(not_accessed+1))                    
        self.profit = list(reversed(self.profit))
    
    def __repr__(self):
        return f"{self.name, self.weight, self.profit, self.accesses, self.start, self.end}"
        

In [190]:
l = [10, 20, 30, 40, 50]

for idx, item in enumerate(reversed(l)):
    print(idx, item) 

l[0:1]

0 50
1 40
2 30
3 20
4 10


[10]

In [211]:
a = feat_for_reuse('a', 5, [1, 2, 3, 4, 5, 6])
a.cal_profit(), print(a.profit)

a = feat_for_reuse('a', 5, [1, 2, 5, 6])
a.cal_profit(), print(a.profit)
# b = feat_for_reuse('b', 5, [2, 3])
# a.get_profit()
# c = feat_for_reuse('c', 5, [3, 4])
# a.get_profit()
# e = feat_for_reuse('e', 5, [5, 7])
# a.get_profit()
# f = feat_for_reuse('f', 5, [6, 7])
# a.get_profit()
# g = feat_for_reuse('g', 5, [7, 8])
# a.get_profit()

# g.is_candidate(6), g.is_candidate(7), g.is_candidate(8), g.is_candidate(9)
# print(g)

[12.5, 25, 20, 15, 10, 5]
[2.5, 5, 3.3333333333333335, 5.0, 10, 5]


(None, None)

In [173]:
class ExclusiveTemporalKnapsack:
    def __init__(self, capacity, end_time, feat_list):
        self.capacity = capacity
        self.feat_list = feat_list
        self.end_time = end_time
    
    def __collect_candidate(self, time_slot):
        # candidate = [feat_for_reuse('0', 0, [0, 0])]
        candidate = []
        for feat in self.feat_list:
            if feat.is_candidate(time_slot):                
                candidate.append(feat)
        return candidate

    def __adjust_unselected(self, time_slot, unpicked_items):
        print(f"Unpicked Items: ", unpicked_items)
        for unpicked in unpicked_items:
            # Change the serach with the dictionary

            # find the index of the picked
            idx = self.feat_list.index(unpicked)
            print("Unpicked!!: ", self.feat_list[idx])
            self.feat_list.remove(unpicked)
            

            
        
    
    def run_tkp(self):
        for time_slot in range(self.end_time, 0, -1):            
            candidate = self.__collect_candidate(time_slot)
            #print(time_slot, candidate)
            print('Time Slot: ', time_slot)
            knapsack = KnapsackProblem(len(candidate), self.capacity, candidate)
            knapsack.solve()
            unpicked_items = knapsack.show_result()
            self.__adjust_unselected(time_slot, unpicked_items)
            #knapsack.show_dp_table()
            print('\n')

            # Adjust unselected items
            
    


In [150]:
# Dynamic Programming: O(n*m), psuedo-polynominal
class KnapsackProblem:

    def __init__(self, num_of_items, capacity, feats):
        self.num_of_items = num_of_items
        self.capacity = capacity
        self.feats = sorted(feats, key=lambda feat : feat.name, reverse=True)
        self.feats.insert(0, feat_for_reuse('0', 0, [0, 0]))
        # self.feats = feats
        self.S = [[0 for _ in range(capacity+1)] for _ in range(num_of_items+1)]
        print(f"{self.num_of_items}, {self.feats}")

    def solve(self):
        # construct the S dynamic programming table
        # O(n*M)
        for i in range(self.num_of_items+1):
            for w in range(self.capacity+1):
                not_taking_item = self.S[i - 1][w]
                taking_item = 0
                if self.feats[i].weight <= w:
                    taking_item = self.feats[i].profit + self.S[i - 1][w - self.feats[i].weight]

                # memoization - we store the sub-results to avoid recalculating the same values
                self.S[i][w] = max(not_taking_item, taking_item)

    def show_result(self):

        print("Total benefit: %.1f" % self.S[self.num_of_items][self.capacity])
        picked_items = []

        w = self.capacity
        for n in range(self.num_of_items, 0, -1):
            if self.S[n][w] != 0 and self.S[n][w] != self.S[n - 1][w]:
                picked_items.append(self.feats[n])
                #print(f"We take item #{n}: {self.feats[n].name}")
                w = w - self.feats[n].weight
        print(f"Picked Items: ", picked_items)

        unpicked_items = []
        for feat in self.feats[1:]:  # skip the '0'
            if feat not in picked_items:
                unpicked_items.append(feat)
        return unpicked_items
    
    def show_dp_table(self):
        for i in range(self.num_of_items+1):
            row = ""
            for w in range(self.capacity+1):
                row += f"{self.S[i][w]:0.1f} "
            print(row)

In [175]:
a = feat_for_reuse('a', 5, [1, 2, 5, 6])  # 2.5
b = feat_for_reuse('b', 5, [2, 3])        # 2.5
c = feat_for_reuse('c', 5, [3, 4])        # 2.5
e = feat_for_reuse('e', 5, [5, 7])        # 1.6
f = feat_for_reuse('f', 5, [6, 7])        # 2.5
g = feat_for_reuse('g', 5, [7, 8])        # 2.5
feats = [a, b, c, e, f, g]
print(feats)

algo = ExclusiveTemporalKnapsack(10, 8, feats)
algo.run_tkp()

[('a', 5, 2.5, [1, 2, 5, 6], 1, 6), ('b', 5, 2.5, [2, 3], 2, 3), ('c', 5, 2.5, [3, 4], 3, 4), ('e', 5, 1.6666666666666667, [5, 7], 5, 7), ('f', 5, 2.5, [6, 7], 6, 7), ('g', 5, 2.5, [7, 8], 7, 8)]
Time Slot:  8
1, [('0', 0, 0.0, [0, 0], 0, 0), ('g', 5, 2.5, [7, 8], 7, 8)]
Total benefit: 2.5
Picked Items:  [('g', 5, 2.5, [7, 8], 7, 8)]
Unpicked Items:  []


Time Slot:  7
3, [('0', 0, 0.0, [0, 0], 0, 0), ('g', 5, 2.5, [7, 8], 7, 8), ('f', 5, 2.5, [6, 7], 6, 7), ('e', 5, 1.6666666666666667, [5, 7], 5, 7)]
Total benefit: 5.0
Picked Items:  [('f', 5, 2.5, [6, 7], 6, 7), ('g', 5, 2.5, [7, 8], 7, 8)]
Unpicked Items:  [('e', 5, 1.6666666666666667, [5, 7], 5, 7)]
Unpicked!!:  ('e', 5, 1.6666666666666667, [5, 7], 5, 7)


Time Slot:  6
2, [('0', 0, 0.0, [0, 0], 0, 0), ('f', 5, 2.5, [6, 7], 6, 7), ('a', 5, 2.5, [1, 2, 5, 6], 1, 6)]
Total benefit: 5.0
Picked Items:  [('a', 5, 2.5, [1, 2, 5, 6], 1, 6), ('f', 5, 2.5, [6, 7], 6, 7)]
Unpicked Items:  []


Time Slot:  5
1, [('0', 0, 0.0, [0, 0], 0, 0), (

In [176]:
a = feat_for_reuse('a', 5, [1, 2, 3, 5, 6])  # 2.5
b = feat_for_reuse('b', 5, [2, 3])        # 2.5
c = feat_for_reuse('c', 5, [3, 4])        # 2.5
e = feat_for_reuse('e', 5, [5, 7])        # 1.6
f = feat_for_reuse('f', 5, [6, 7])        # 2.5
g = feat_for_reuse('g', 5, [7, 8])        # 2.5
feats = [a, b, c, e, f, g]
print(feats)

algo = ExclusiveTemporalKnapsack(10, 8, feats)
algo.run_tkp()

[('a', 5, 3.3333333333333335, [1, 2, 3, 5, 6], 1, 6), ('b', 5, 2.5, [2, 3], 2, 3), ('c', 5, 2.5, [3, 4], 3, 4), ('e', 5, 1.6666666666666667, [5, 7], 5, 7), ('f', 5, 2.5, [6, 7], 6, 7), ('g', 5, 2.5, [7, 8], 7, 8)]
Time Slot:  8
1, [('0', 0, 0.0, [0, 0], 0, 0), ('g', 5, 2.5, [7, 8], 7, 8)]
Total benefit: 2.5
Picked Items:  [('g', 5, 2.5, [7, 8], 7, 8)]
Unpicked Items:  []


Time Slot:  7
3, [('0', 0, 0.0, [0, 0], 0, 0), ('g', 5, 2.5, [7, 8], 7, 8), ('f', 5, 2.5, [6, 7], 6, 7), ('e', 5, 1.6666666666666667, [5, 7], 5, 7)]
Total benefit: 5.0
Picked Items:  [('f', 5, 2.5, [6, 7], 6, 7), ('g', 5, 2.5, [7, 8], 7, 8)]
Unpicked Items:  [('e', 5, 1.6666666666666667, [5, 7], 5, 7)]
Unpicked!!:  ('e', 5, 1.6666666666666667, [5, 7], 5, 7)


Time Slot:  6
2, [('0', 0, 0.0, [0, 0], 0, 0), ('f', 5, 2.5, [6, 7], 6, 7), ('a', 5, 3.3333333333333335, [1, 2, 3, 5, 6], 1, 6)]
Total benefit: 5.8
Picked Items:  [('a', 5, 3.3333333333333335, [1, 2, 3, 5, 6], 1, 6), ('f', 5, 2.5, [6, 7], 6, 7)]
Unpicked Items: 